# Урок 5. Part-of-Speech разметка, NER, извлечение отношений

## Домашнее задание

### Задание 1. Написать теггер на данных с русским языком
1. проверить `UnigramTagger`, `BigramTagger`, `TrigramTagger` и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы

In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pyconll

In [3]:
#!git clone https://github.com/named-entity/nltk4russian.git
#!python /content/nltk4russian/setup.py install
#import nltk4russian
#from nltk4russian.tagger import PMContextTagger

In [4]:
!mkdir datasets

In [5]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-01-30 06:35:44--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-01-30 06:35:44 ERROR 404: Not Found.

--2023-01-30 06:35:45--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14704579 (14M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-dev.conllu’

./datasets/ru_synta 100%[===================

In [6]:
full_train = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-dev.conllu')
full_test = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-dev.conllu')

In [7]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Алгоритм NOUN
, PUNCT
от ADP
имени NOUN
учёного NOUN
аль PART
- PUNCT
Хорезми PROPN
, PUNCT
- PUNCT
точный ADJ
набор NOUN
инструкций NOUN
, PUNCT
описывающих VERB
порядок NOUN
действий NOUN
исполнителя NOUN
для ADP
достижения NOUN
результата NOUN
решения NOUN
задачи NOUN
за ADP
конечное ADJ
время NOUN
. PUNCT

В ADP
старой ADJ
трактовке NOUN
вместо ADP
слова NOUN
" PUNCT
порядок NOUN
" PUNCT
использовалось VERB
слово NOUN
" PUNCT
последовательность NOUN
" PUNCT
, PUNCT
но CCONJ
по ADP
мере NOUN
развития NOUN
параллельности NOUN
в ADP
работе NOUN
компьютеров NOUN
слово NOUN
" PUNCT
последовательность NOUN
" PUNCT
стали VERB
заменять VERB
более ADV
общим ADJ
словом NOUN
" PUNCT
порядок NOUN
" PUNCT
. PUNCT



In [8]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [9]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(str(token.form)) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 144
Наибольшая длина токена 30


In [10]:
all_train_texts = [' '.join(str(token.form) for token in sent) for sent in full_train]
all_test_texts = [' '.join(str(token.form) for token in sent) for sent in full_test]

all_train_labels = [' '.join(str(token.upos) for token in sent) for sent in full_train]
all_test_labels = [' '.join(str(token.upos) for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Алгоритм , от имени учёного аль - Хорезми , - точный набор инструкций , описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время .
В старой трактовке вместо слова " порядок " использовалось слово " последовательность " , но по мере развития параллельности в работе компьютеров слово " последовательность " стали заменять более общим словом " порядок " .
Это связано с тем , что работа каких-то инструкций алгоритма может быть зависима от других инструкций или результатов их работы .
Таким образом , некоторые инструкции должны выполняться строго после завершения работы инструкций , от которых они зависят .
Независимые инструкции или инструкции , ставшие независимыми из-за завершения работы инструкций , от которых они зависят , могут выполняться в произвольном порядке , параллельно или одновременно , если это позволяют используемые процессор и операционная система .
Ранее часто писали " алгорифм " , сейчас такое написание используется редко , но , т

In [11]:

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [12]:
unigram_tagger = UnigramTagger(fdata_train)
bigram_tagger = BigramTagger(fdata_train)#, backoff=unigram_tagger) 
trigram_tagger = TrigramTagger(fdata_train)

In [13]:
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

<ipython-input-13-fd8c468215e5>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.970551468194544

In [14]:
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

<ipython-input-14-d56645bf4924>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', None),
 ('многих', None),
 ('вариантах', None),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.9604661761833453

In [15]:
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))

<ipython-input-15-ed79a29e8596>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', None),
 ('многих', None),
 ('вариантах', None),
 ('(', None),
 ('самые', None),
 ('ранние', None),
 ('из', None),
 ('них', None),
 ('почти', None),
 ('на', 'ADP'),
 ('сто', None),
 ('лет', None),
 ('старше', None),
 (')', None),
 ('и', None),
 ('восходит', None),
 ('к', None),
 ('ещё', None),
 ('более', None),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', None),
 ('.', 'PUNCT')]

0.8668663324435184

Точность выше при использовании unigram_tagger, но и bigram_tagger показывает хорошее качество. Использование trigram_tagger показало качество хуже

Посмотрим на точность при комбинации теггеров

In [16]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 

tag.evaluate(fdata_test) 


<ipython-input-16-9b1f8847150d>:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tag.evaluate(fdata_test)


0.9828764893547757

При комбинации теггеров выходит чуть лучше

Обучим модель логистической регрессии

In [17]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(str(tok[0]))
        train_label.append('NO_TAG' if str(tok[1]) is None else str(tok[1]))
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(str(tok[0]))
        test_label.append('NO_TAG' if str(tok[1]) is None else str(tok[1]))

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [20]:
test_enc_labels = le.transform(test_label)

In [21]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'None', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB',
       'X'], dtype='<U5')

In [22]:
hvectorizer = HashingVectorizer(ngram_range=(1, 2), analyzer='char', n_features=500)
hvectorizerw = HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=500)

In [23]:
X_train = hvectorizer.fit_transform(train_tok)
X_trainw = hvectorizerw.fit_transform(train_tok)

In [24]:
X_test = hvectorizer.transform(test_tok)
X_testw = hvectorizerw.transform(test_tok)

In [25]:
X_train.shape

(153590, 500)

In [26]:
X_trainw.shape

(153590, 500)

In [27]:
%%time
#Обучим модель , когда анализируются буквы
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

CPU times: user 18.9 s, sys: 103 ms, total: 19 s
Wall time: 18.9 s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [28]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8371313236538838

In [29]:
# Посмотрим модель с анализом слов
lr_w = LogisticRegression(random_state=0, max_iter=100)

In [30]:
lr_w.fit(X_trainw, train_enc_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [31]:
predw = lr_w.predict(X_testw)
accuracy_score(test_enc_labels, predw)

0.3402500162771014

Получилось хуже

Попробуем 

In [32]:
countvectorizer = CountVectorizer(ngram_range=(1, 2), analyzer='char')
countvectorizerw = CountVectorizer(ngram_range=(1, 2), analyzer='word')

In [33]:
X_train = countvectorizer.fit_transform(train_tok)
X_trainw = countvectorizerw.fit_transform(train_tok)

In [34]:
X_test = countvectorizer.transform(test_tok)
X_testw = countvectorizerw.transform(test_tok)

In [35]:
X_train.shape

(153590, 1448)

In [36]:
X_trainw.shape

(153590, 34409)

In [37]:
%%time
#Обучим модель , когда анализируются буквы
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

CPU times: user 29.5 s, sys: 21 s, total: 50.6 s
Wall time: 27.8 s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [38]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8907611172602383

In [39]:
# Посмотрим модель с анализом слов
lr_w = LogisticRegression(random_state=0, max_iter=100)

In [40]:
lr_w.fit(X_trainw, train_enc_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [41]:
predw = lr_w.predict(X_testw)
accuracy_score(test_enc_labels, predw)

0.6872192200013022

In [42]:
# На данном тексте лучше всего себя показали unigram_tagger и  bigram_tagger

### Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы:
    - передаём в сетку токен и его соседей.
    - передаём в сетку только токен.
    - свой вариант.
3. сравнить свои реализованные подходы на качество — вывести `precision/recall/f1_score`.ipynb_checkpoints/

In [43]:
!python -m venv env 
#.\env\Scripts\activate.bat
!pip install deeppavlov
!python -m deeppavlov install squad_bert

!python -m deeppavlov install ner_ontonotes

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.8-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: ['/content/env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.7/441.7 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4

In [44]:
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_rus_bert, download=True, install=True)

2023-01-30 06:40:30.75 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz
100%|██████████| 1.44G/1.44G [00:54<00:00, 26.2MB/s]
2023-01-30 06:41:25.895 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_torch
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_torch


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [45]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 KB 3.8 MB/s eta 0:00:00


In [46]:
import corus

In [47]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-01-30 06:42:24--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.66MB/s    in 1.1s    

2023-01-30 06:42:26 (1.66 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [48]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [49]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='055',
    text='Москве потребовался год на поиски кандидата в генсеки "газовой ОПЕК"\r\nПравительство России после почти года поисков определилось со своим кандидатом на пост генерального секретаря Форума стран - экспортеров газа (ФСЭГ), который в прессе часто называют "газовой ОПЕК". Газета "Коммерсант" пишет, что российским кандидатом на пост главы ФСЭГ будет Леонид Бохановский, вице-президент компании "Стройтрансгаз". Ранее все потенциальные претенденты от России отказались бороться за предлагаемую им должность.\r\n\r\nГазета пишет, что своего кандидата на пост генсека ФСЭГ Россия представит в Дохе 9 ноября. Данная должность является ключевой для организации, поскольку занимающий этот пост человек должен обеспечивать разработку бюджета и программу форума, а также управлять его работой. Занять этот пост на два года российский кандидат сможет лишь в том случае, если его кандидатура будет одобрена единогласным решением всех министров энергетики стран-участниц форума.

In [50]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from razdel import tokenize
import pandas as pd

In [52]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [53]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [54]:
df_words['tag'].value_counts()

OUT         218908
PER          21190
ORG          13631
LOC           4565
GEOPOLIT      4326
MEDIA         2480
Name: tag, dtype: int64

In [55]:
df_words.head(2)

,word,tag
0,Д,PER
1,.,PER


In [56]:
df_words.shape

(265100, 2)

In [57]:
doc = df_words.word.values.tolist()

In [58]:
doc_str = ' '.join(doc)

In [59]:
deeppavlov_ner([doc_str[:511]]) 

[[['Д',
   '.',
   'Медведев',
   'назначил',
   'В',
   '.',
   'Бондарева',
   'на',
   'должность',
   'главкома',
   'ВВС',
   'РФ',
   '.',
   'Президент',
   'России',
   'Дмитрий',
   'Медведев',
   'подписал',
   'указ',
   '"',
   'О',
   'главнокомандующем',
   'Военно',
   '-',
   'воздушными',
   'силами',
   '"',
   ',',
   'передает',
   'пресс',
   '-',
   'служба',
   'Кремля',
   '.',
   'В',
   'документе',
   ',',
   'в',
   'частности',
   'говорится',
   ':',
   '"',
   'Назначить',
   'генерал',
   '-',
   'майора',
   'Бондарева',
   'Виктора',
   'Николаевича',
   'главнокомандующим',
   'Военно',
   '-',
   'воздушными',
   'силами',
   ',',
   'освободив',
   'его',
   'от',
   'должности',
   'начальника',
   'Главного',
   'штаба',
   '-',
   'первого',
   'заместителя',
   'главнокомандующего',
   'Военно',
   '-',
   'воздушными',
   'силами',
   '"',
   '.',
   'Напомним',
   ',',
   '27',
   'апреля',
   '2012',
   'г',
   '.',
   'Д',
   '.',
   'Медвед

#### Spacy

In [60]:
import requests
from bs4 import BeautifulSoup
import re


def url_to_string(url, headers):
    res = requests.get(url, headers=headers)
    html = res.text
    print(res)
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en"
    }

url = 'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news'
document = url_to_string(url, headers)


<Response [200]>


In [61]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-py3-none-any.whl size=98072933 sha256=a57bee858971c89df3aa1f033599339d3ad22c89a956a8cf5c705fb2404f9ff9
  Stored in directory: /root/.cache/pip/wheels/31/1f/21/52bc46cea1e50c10a3b246d5d266455b9a0e77c380bb7ac928
Successfully built en-core-web-md


In [62]:
from spacy import displacy
!python -m spacy download en_core_web_md
## import en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()

2023-01-30 06:43:30.736259: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 2.2.0
    Uninstalling en-core-web-md-2.2.0:
      Successfully uninstalled en-core-web-md-2.2.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [63]:
doc = nlp(document)
displacy.render(doc, style="dep", jupyter=True) # (1)

<img src='17_5_files/outputframe_5_.png'>

In [64]:
displacy.render(doc, style="ent", jupyter=True) # (2)

In [65]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [66]:
from sklearn import model_selection, preprocessing

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
# encoder = preprocessing.LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# valid_y = encoder.transform(valid_y)

In [67]:
encoder_one = preprocessing.OneHotEncoder(sparse=False)
train_y = encoder_one.fit_transform(train_y.values.reshape(-1, 1))
valid_y = encoder_one.transform(valid_y.values.reshape(-1, 1))

In [68]:
train_x.apply(len).max(axis=0)

55

In [69]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [70]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [71]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 2),
    output_sequence_length=seq_len,)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [72]:
len(vectorize_layer.get_vocabulary())

29887

In [73]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [74]:
mmodel = modelNER()

In [75]:
# mmodel.build((16, 1))
# mmodel.summary()

In [76]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics='accuracy')

In [77]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12427/12427 [==============================] - 302s 24ms/step - loss: 0.2949 - accuracy: 0.9138 - val_loss: 0.2109 - val_accuracy: 0.9388


In [78]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00


In [79]:
import tensorflow_addons as tfa
f1 = tfa.metrics.F1Score(num_classes=6, threshold=0.5, average='macro') #num_classes
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[f1])

In [80]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12427/12427 [==============================] - 319s 26ms/step - loss: 0.1260 - f1_score: 0.8837 - val_loss: 0.2437 - val_f1_score: 0.8394


In [81]:
metric = tfa.metrics.FBetaScore(num_classes=6, beta=2.0, threshold=0.5)
metric.update_state(valid_y, pred)
result = metric.result()
result.numpy()

InvalidArgumentError: ignored

Precision, Recall

In [82]:
mmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [83]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12427/12427 [==============================] - 292s 23ms/step - loss: 0.1111 - recall: 0.9649 - precision: 0.9666 - val_loss: 0.2655 - val_recall: 0.9386 - val_precision: 0.9406


In [ ]:
# from keras import backend as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# mmodel.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#               metrics=['acc',f1_m,precision_m, recall_m])